# StationSim - Density version
    author: P. Ternes
    created: 20/08/2020

In this text we track changes made in the StationSim_gcs model to emulate the data from the Grand Central Station using a local geographic density as a interaction parameter.


## Model code

The StationSim density model code is the StationSim folder and is linked ['here.'](../../stationsim/stationsim_density_model.py).

## Manly changes

* Create a local geographic density parameter;
* Create a get_local_density method;
* Update the speed using the local geographic density;
* Update the direction of movement using the local geographic density;
* Remove some code parts;

## StationSim Grand Central Terminal model

The StationSim density version is based on StationSim Grand Central Terminal version.

To know more about the StationSim_gcs you can access [`this notebook`](../gcs_experiments/StationSim_GrandCentral_version.ipynb) jupyter notebook. 

## Local Density

In this model the interactions between the agents are controlled by one density parameter.
This parameter mesures the local geographic density around each agent and is used to update the speed and direction of movemment. 

The local geographic density is defined as the number of neighbors agents (n) divided by the considered area (A).

$$\rho = \frac{n}{A}.$$

For this area the maximum possible density is

$$\rho_{max} = \frac{N}{A},$$

where N is the maximum possible number of agents inside the area A. Then, the normalized local geographic density is:

$$\rho^* = \dfrac{\rho}{\rho_{max}} = \dfrac{n}{N}.$$

Consider the format of each agent to be a disk of radius $r$. The maximum possible number of agents insire a radius $R$ from the central agent is the total available area divided by the area that each agent:

$$N = \dfrac{\pi (R^2 - r^2)}{\pi r^2},$$

finally, the normalized local geographic density can be rewritten as:

$$\rho^* = n \dfrac{r^2}{(R^2 - r^2)}.$$


To obtain the normalized local geographic density it is necessary call the <b>get_local_density</b> method:


In [ ]:
def get_local_density(self):
    '''
     A function to determine the normalized
     local geographic density around each agent. 
    '''

    state = self.model.get_state('location2D')
    self.model.tree = cKDTree(state)
    neighbouring_agents = self.model.tree.query_ball_point(self.location, self.local_density_radius)
    self.local_density = len(neighbouring_agents) * self.size**2 / 
                                                (self.local_density_radius**2 - self.size**2) # density between 0-1

## Speed

Each agent receives a speed value ($v$) when created. This value represents the maximum possible speed that each agent can walk.

If the agent is walking without any other agent around, then the agent can walk with the maximum speed. If the agent is in a very crowed situation, then the agent walks more slowly. Therefore, the speed is a function of the local densitity. In each time step the update value for speed ($v'$) is:

$$ v' = v(1 - \rho^*)$$

## Direction of movement

Each agent receives desired location when created.

If the agent is walking without any other agent around, then the agent can walk in direction to the desired location. If the agent is in a very crowed situation, then the agent can chose different directions to give the next step. So, the change in the movement direction is related with the local density. 

To update the direction vector $(x,y)$, first a rotation angle $(\theta)$ is draw from a gaussian distribution with mean of 0.0 and standart deviation of $(180^{\circ}\times\rho^*)$. If the angle is positive, then the movement direction is rotated in an anticlockwise direction, and if the angle is negative, then the movement direction is rotated in a clockwise direction. The new direction $(x',y')$ after rotation is obtained using a rotation matrix:

$$\begin{bmatrix}x'\\ y'\end{bmatrix} = 
\begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}
\begin{bmatrix}x\\ y\end{bmatrix} $$


## Removed code

All collision-related parts of the code are not required in this model and can be removed. This include the following methods:

* get_collisionTime2Agents
* get_collisionTimeWall
* get_collisionTable
* get_wiggleTable
* set_wiggle

#### Preliminar results

Both, classical and Grand Central versions, are working with the new collision definition. The basic experiments for this model can be found at [`density_experiments`](density_experiments.ipynb).